In [1]:
#Mcallen
#San Antonio
#Edinburg
#Austin
#Dallas
#San Marcos
#Corpus Christi
#Weslaco
#Boerne
#Houston

### Imports

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import urllib.parse

### HTTP Request

In [2]:
#store website in a variable
website = 'https://www.trulia.com/TX/Weslaco/'

In [3]:
#set up selenium webdriver
driver = webdriver.Chrome()

#get the webpage using selenium

try:
    # Get the webpage using Selenium
    driver.get(website)
    time.sleep(5)  # Give the page some time to load

    # Scroll down to load more content
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Adjust the delay based on the page loading speed

    # Get the updated page source after scrolling
    updated_html = driver.page_source

    # Create a BeautifulSoup object using the updated HTML
    soup = BeautifulSoup(updated_html, 'html.parser')

    # Extract the desired results with data-testid
    result_container = soup.find_all(attrs={'data-testid': True})


    # Retrieve anchor tags into a list and establish base URL
    url_part_1 = 'https://www.trulia.com'
    url_part_2 = []

    for item in result_container:
        for link in item.find_all('div', {'data-testid': 'property-card-details'}):
            url_part_2.append(link.find('a').get('href'))

finally:
    driver.quit()  # Close the browser window when done

In [4]:
# Remove spaces and duplicates from list
url_only = [x.strip(' ') for x in url_part_2]
url_only = list(dict.fromkeys(url_only))
len(url_only)

40

In [5]:
# Concatenate URL
url_joined = []
for link_2 in url_only:
    url_joined.append(urllib.parse.urljoin(url_part_1,link_2))

In [6]:
url_joined

['https://www.trulia.com/home/1810-hope-ave-weslaco-tx-78596-345154449',
 'https://www.trulia.com/home/1812-hope-ave-weslaco-tx-78596-345154450',
 'https://www.trulia.com/home/1808-hope-ave-weslaco-tx-78596-345150853',
 'https://www.trulia.com/home/1806-hope-ave-weslaco-tx-78596-345150680',
 'https://www.trulia.com/home/1804-hope-ave-weslaco-tx-78596-345150681',
 'https://www.trulia.com/home/1802-hope-ave-weslaco-tx-78596-345149989',
 'https://www.trulia.com/home/1706-hope-ave-weslaco-tx-78596-345149740',
 'https://www.trulia.com/home/1710-hope-ave-weslaco-tx-78596-345149741',
 'https://www.trulia.com/home/1708-hope-ave-weslaco-tx-78596-345149592',
 'https://www.trulia.com/home/1704-hope-ave-weslaco-tx-78596-345149448',
 'https://www.trulia.com/home/4016-steffy-dr-weslaco-tx-78599-66967913',
 'https://www.trulia.com/home/2102-bald-cypress-dr-weslaco-tx-78596-80076771',
 'https://www.trulia.com/home/W-Mile-10-N-Weslaco-TX-78596-345106052',
 'https://www.trulia.com/home/312-Sparrow-Rd-We

### Get data from links

In [7]:
# address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

In [8]:
first_link = url_joined[0]
print(first_link)

https://www.trulia.com/home/1810-hope-ave-weslaco-tx-78596-345154449


### Get request and Soup Object

In [9]:
response = requests.get(first_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [10]:
soup = BeautifulSoup(response.content, 'html.parser')

In [1]:
# Commenting this step out to avoid copyright
#soup

#### Put all of the attributes together and loop through results from page 1


In [15]:
address = []
bedrooms = []
bathrooms = []
area = []
year_built = []
parking = []
price = []

# Loop through all joined links
for link in url_joined:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
    
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Address
    try:
        address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    # Bedrooms  
    try:
        bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
    except:
        bedrooms.append('')
        
    # Bathrooms
    try:
        bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
    except:
        bathrooms.append('')
        
    # Square Feet
    try:
        area.append(soup.find('li', {'data-testid':'floor'}).get_text())
    except:
        area.append('')
    
    # Year Built
    try:
        year_built.append(soup.find('div',string='Year Built').findNext('span').get_text())
        year_built = [x.strip('Year Built: ') for x in year_built] 
    except:
        year_built.append('')
    
    # Parking
    try:
        parking.append(soup.find(string='Parking').findNext('div').get_text())
    except:
        parking.append('')
    
    # Price
    try:
        price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('') 
    
    # Create a dictionary with results
    output = {'Address': address, 'Bathrooms': bathrooms, 'Area':area, 'Year Built':year_built, 
              'Parking':parking, 'Price':price}
    
output

{'Address': ['1810 Hope Ave',
  '1812 Hope Ave',
  '1808 Hope Ave',
  '1806 Hope Ave',
  '1804 Hope Ave',
  '1802 Hope Ave',
  '1706 Hope Ave',
  '1710 Hope Ave',
  '1708 Hope Ave',
  '1704 Hope Ave',
  '4016 Steffy Dr',
  '2102 Bald Cypress Dr',
  'W  Mile 10 N',
  '312 Sparrow Rd',
  '2501 Sage Dr',
  '1702 Hope Ave',
  '1610 Hope Ave',
  '1606 Hope Ave',
  '1608 Hope Ave',
  'Camille St',
  '1604 Hope Ave',
  '1602 Hope Ave',
  'Zelma St',
  'Emily St',
  'Emily Ln',
  'Bridget St',
  '1603 Hope Ave',
  '1605 Hope Ave',
  '1601 Hope Ave',
  '1607 Hope Ave',
  '1609 Hope Ave',
  '1701 Hope Ave',
  '2006 Mexico Ave',
  'Bahia Plan in Texas Heights by Esperanza Homes',
  '704 Juan Seguin St',
  'Solano Plan in Texas Heights by Esperanza Homes',
  '2005 Alexandra Dr',
  '3816 Steffy Dr',
  '3400 N  Westgate Dr',
  '3817 Santo Cielo St'],
 'Bathrooms': ['2 Baths',
  '2 Baths',
  '2 Baths',
  '2 Baths',
  '2 Baths',
  '2 Baths',
  '2 Baths',
  '2 Baths',
  '2 Baths',
  '2 Baths',
  '2 Bat

In [17]:
# Create dataframe
df = pd.DataFrame(output)
df

,Address,Bathrooms,Area,Year Built,Parking,Price
0,1810 Hope Ave,2 Baths,"1,275 sqft",2024,No Info,"$209,000"
1,1812 Hope Ave,2 Baths,"1,125 sqft",2024,No Info,"$199,000"
2,1808 Hope Ave,2 Baths,"1,125 sqft",2024,No Info,"$199,000"
3,1806 Hope Ave,2 Baths,"1,275 sqft",2024,No Info,"$209,000"
4,1804 Hope Ave,2 Baths,"1,125 sqft",2024,No Info,"$199,000"
5,1802 Hope Ave,2 Baths,"1,275 sqft",2024,No Info,"$209,000"
6,1706 Hope Ave,2 Baths,"1,125 sqft",2024,No Info,"$199,000"
7,1710 Hope Ave,2 Baths,"1,125 sqft",2024,No Info,"$199,000"
8,1708 Hope Ave,2 Baths,"1,275 sqft",2024,No Info,"$209,000"
9,1704 Hope Ave,2 Baths,"1,275 sqft",2024,No Info,"$209,000"


In [18]:
# Add location to dataframe
df['Location'] = 'Weslaco'
df

,Address,Bathrooms,Area,Year Built,Parking,Price,Location
0,1810 Hope Ave,2 Baths,"1,275 sqft",2024,No Info,"$209,000",Weslaco
1,1812 Hope Ave,2 Baths,"1,125 sqft",2024,No Info,"$199,000",Weslaco
2,1808 Hope Ave,2 Baths,"1,125 sqft",2024,No Info,"$199,000",Weslaco
3,1806 Hope Ave,2 Baths,"1,275 sqft",2024,No Info,"$209,000",Weslaco
4,1804 Hope Ave,2 Baths,"1,125 sqft",2024,No Info,"$199,000",Weslaco
5,1802 Hope Ave,2 Baths,"1,275 sqft",2024,No Info,"$209,000",Weslaco
6,1706 Hope Ave,2 Baths,"1,125 sqft",2024,No Info,"$199,000",Weslaco
7,1710 Hope Ave,2 Baths,"1,125 sqft",2024,No Info,"$199,000",Weslaco
8,1708 Hope Ave,2 Baths,"1,275 sqft",2024,No Info,"$209,000",Weslaco
9,1704 Hope Ave,2 Baths,"1,275 sqft",2024,No Info,"$209,000",Weslaco


In [19]:
df.to_excel('dataframe_weslaco.xlsx', index=False)